In [282]:
import numpy as np 
import scipy.io as sio
import matplotlib.pyplot as plt 
import os
from tqdm import tqdm, notebook
import cartopy
import xarray as xr

In [283]:
# era5_file = '/localdrive/drive10/jj/mdtf/wkdir/MDTF_ERA5.ALL.DEG15.001_1950_2019/etc_composites/tmp/RUNDIR/tmprun/read_tmprun/tmprun_1950.mat'
era5_file = '/localdrive/drive10/jj/mdtf/wkdir/MDTF_ERA5.ALL.DEG15.001_1950_1950/etc_composites/tmp/RUNDIR/tmprun/read_tmprun/tmprun_1950.mat'
test_file = '/localdrive/drive10/jj/mdtf/wkdir/MDTF_ERA5.ALL.DEG15.001_1950_2019/etc_composites/tmp/RUNDIR/tmprun/read_tmprun/tmprun_1950.mat'
gfdl_file = '/localdrive/drive10/jj/mdtf/wkdir/MDTF_GFDL.EXP1.2PM.001_2008_2012/etc_composites/tmp/RUNDIR/tmprun/read_tmprun/tmprun_2008.mat'
erai_file = '/localdrive/drive10/jj/mdtf/wkdir/MDTF_ERAI.EXP1.TEST.001_2008_2012/etc_composites/tmp/RUNDIR/tmprun/read_tmprun/tmprun_2008.mat'

In [284]:
e5Data = sio.loadmat(era5_file)['cyc']
testData = sio.loadmat(test_file)['cyc']
gData = sio.loadmat(gfdl_file)['cyc']
eiData = sio.loadmat(erai_file)['cyc']

In [ ]:
# data = eiData
data = e5Data
# data = gData

var_list = ['fulllat', 'fulllon', 'fullslp', 'fullyr', 'fullmon', 'fullday', 'fullhr']

in_data = {}
for var in var_list:
    in_data[var] = []
    
for i in notebook.tqdm(range(data['fulllat'].shape[1])):
    for var in var_list:
        in_data[var].extend(np.squeeze(np.squeeze(data[var])[i]))
        
for var in var_list:
    in_data[var] = np.array(in_data[var])

On November 25th, 1950, there was a storm that did significant damage to the New York area.

This track is apparently missing from my tracks.

I have to diagnose this problem.

In [ ]:
lon = in_data['fulllon']
lon[lon > 180] -= 360 

In [ ]:
ind = (in_data['fullyr'] == 1950) & (in_data['fullmon'] == 11) & (in_data['fullday'] == 25)
# ind = (in_data['fullyr'] == 1950)

In [ ]:
plt.figure()
ax = plt.subplot(111, projection=cartopy.crs.PlateCarree())
ax.plot(in_data['fulllon'][ind], in_data['fulllat'][ind], 'r*')
ax.coastlines()
ax.set_extent([-180, 180, -90, 90])
plt.show()

Lets check if the storm is visible from my SLP input data

In [ ]:
slp_file = '/localdrive/drive10/jj/mdtf/inputdata/model/ERA5.ALL.DEG15.001/6hr/ERA5.ALL.DEG15.001.SLP.6hr.nc'
ds = xr.open_dataset(slp_file)
select = ds.sel(time='1950-11-25')

In [ ]:
ny_cdt = [-74, 40]
lon_div = 50
lat_div = 40
extent = [ny_cdt[0]-lon_div, ny_cdt[0]+lon_div, ny_cdt[1]-lat_div, ny_cdt[1]+lat_div] #40.7128° N, 74.0060° W
 
plt.figure(figsize=(12,12))
ax=plt.subplot(2,2,1, projection=cartopy.crs.PlateCarree())
select.SLP.isel(time=0).plot(ax=ax)
ax.coastlines()
ax.set_extent(extent)

ax=plt.subplot(2,2,2, projection=cartopy.crs.PlateCarree())
select.SLP.isel(time=1).plot(ax=ax)
ax.coastlines()
ax.set_extent(extent)

ax=plt.subplot(2,2,3, projection=cartopy.crs.PlateCarree())
select.SLP.isel(time=2).plot(ax=ax)
ax.coastlines()
ax.set_extent(extent)

ax=plt.subplot(2,2,4, projection=cartopy.crs.PlateCarree())
select.SLP.isel(time=3).plot(ax=ax)
ax.coastlines()
ax.set_extent(extent)

plt.show()


Lets read in the raw ERA5 SLP data to track the cyclones

In [ ]:
file = '/localdrive/drive6/era5/data/six_hrly/data_1_5deg/msl/msl_1950_6hrly.nc'
ds = xr.open_dataset(file)
select = ds.sel(time='1950-11-25')

In [ ]:
lon_div = 50
lat_div = 40
extent = [-74-lon_div, -74+lon_div, 40-lat_div, 40+lat_div] #40.7128° N, 74.0060° W
 
plt.figure(figsize=(12,12))
ax=plt.subplot(2,2,1, projection=cartopy.crs.PlateCarree())
select.msl.isel(time=0).plot(ax=ax)
ax.coastlines()
ax.set_extent(extent)

ax=plt.subplot(2,2,2, projection=cartopy.crs.PlateCarree())
select.msl.isel(time=1).plot(ax=ax)
ax.coastlines()
ax.set_extent(extent)

ax=plt.subplot(2,2,3, projection=cartopy.crs.PlateCarree())
select.msl.isel(time=2).plot(ax=ax)
ax.coastlines()
ax.set_extent(extent)

ax=plt.subplot(2,2,4, projection=cartopy.crs.PlateCarree())
select.msl.isel(time=3).plot(ax=ax)
ax.coastlines()
ax.set_extent(extent)
plt.show()

Lets now check the dumped centers from the RUN

In [ ]:
import pandas as pd

In [ ]:
dumped_file = '/localdrive/drive10/jj/mdtf/wkdir/MDTF_ERA5.ALL.DEG15.001_1950_1950/etc_composites/tmp/RUNDIR/tmprun/out_tmprun/tmprun/mcms_tmprun_1950_dumped_centers.txt'
centers_file = '/localdrive/drive10/jj/mdtf/wkdir/MDTF_ERA5.ALL.DEG15.001_1950_1950/etc_composites/tmp/RUNDIR/tmprun/out_tmprun/tmprun/mcms_tmprun_1950_centers.txt'


In [ ]:
dumped_df = pd.read_csv(dumped_file,sep='\s+', header=None)
centers_df = pd.read_csv(centers_file,sep='\s+', header=None)
dumped_df.insert(16, 'new-col', 0)
centers_df.insert(16, 'new-col', 1)

main_df = pd.concat([dumped_df, centers_df])

df = main_df.iloc[:, [0, 1, 2, 3, 5, 6, 8, 15, 16]].copy()
df.columns = ['yy', 'mm', 'dd', 'hh', 'lat', 'lon', 'slp', 'usi', 'type']
df.lat = 90. - df.lat/100. 
df.lon = df.lon/100.
df['lon'][df.lon > 180] = df.lon[df.lon > 180] - 360

print(df.shape)

In [ ]:
dumped_ind = (df.yy == 1950) & (df.mm == 11) & (df.dd == 25) & (df.type == 0)
print(np.sum(dumped_ind))
centers_ind = (df.yy == 1950) & (df.mm == 11) & (df.dd == 25) & (df.type == 1)
print(np.sum(centers_ind))

plt.figure()
ax = plt.subplot(111, projection=cartopy.crs.PlateCarree())
ax.plot(df.lon[dumped_ind], df.lat[dumped_ind], 'b*', label='dumped')
ax.plot(df.lon[centers_ind], df.lat[centers_ind], 'r.', label='centers')
ax.coastlines()
ax.set_extent(extent)
plt.legend(loc=0)
plt.show()

Looks like the centers are detected in the centers file.

Now we have to check if it is also there in tracks file.

In [ ]:
tracks_file = '/localdrive/drive10/jj/mdtf/wkdir/MDTF_ERA5.ALL.DEG15.001_1950_1950/etc_composites/tmp/RUNDIR/tmprun/out_tmprun/tmprun/mcms_tmprun_1950_tracks.txt'

In [ ]:
main_df = pd.read_csv(tracks_file, sep='\s+', header=None)

df = main_df.iloc[:, [0, 1, 2, 3, 5, 6, 8, 15]].copy()
df.columns = ['yy', 'mm', 'dd', 'hh', 'lat', 'lon', 'slp', 'usi']
df.lat = 90. - df.lat/100. 
df.lon = df.lon/100.
df['lon'][df.lon > 180] = df.lon[df.lon > 180] - 360

In [ ]:
ind = (df.yy == 1950) & (df.mm == 11) & (df.dd == 25)

plt.figure()
ax = plt.subplot(111, projection=cartopy.crs.PlateCarree())
ax.plot(df.lon[ind], df.lat[ind], 'b*', label='tracked')
ax.coastlines()
ax.set_extent(extent)
plt.legend(loc=0)
plt.show()

Lets check the USI values for the New York tracks

In [ ]:
ind = (df.yy == 1950) & (df.mm == 11) & (df.dd == 25) & (df.lat > ny_cdt[1]-10) & \
    (df.lat < ny_cdt[1]+10) & (df.lon > ny_cdt[0]-10) & (df.lon < ny_cdt[0]+10)

i_ind = df.usi[ind].values
i_ind = i_ind[2]

usi_ind = (df.usi == i_ind)
x = df[usi_ind]
# print(len(df.hh[usi_ind]))

In [ ]:
import datetime as dt
from datetime import date
dates = [dt.datetime(iyy, imm, idd, ihh) for iyy,imm,idd,ihh in zip(df.yy[usi_ind], df.mm[usi_ind], df.dd[usi_ind], df.hh[usi_ind])]
full_dates = [date.toordinal(date(iyy, imm, idd))+360+ihh/24. for iyy,imm,idd,ihh in zip(df.yy[usi_ind], df.mm[usi_ind], df.dd[usi_ind], df.hh[usi_ind])]

In [ ]:
print(full_dates)

In [ ]:
print((dates[1]-dates[0]).total_seconds()/3600)
print((full_dates[1] - full_dates[0])*24)

In [ ]:
datet.

In [ ]:
hh = np.array(df.hh[usi_ind], dtype=int)